# Ref
https://huggingface.co/docs/transformers/tasks/token_classification

# Constant

In [1]:
# TRAINED_MODEL_PATH_1 = "/kaggle/input/e099-original-fake-data-deberta-v3-large"
TRAINED_MODEL_PATH_1 = "pll_data_detection/trained_models/e072-fix-tokenize"

TRAINING_MODEL_PATHS = [
    TRAINED_MODEL_PATH_1
]

# weights = [1, 1] # average
weights = [1] # average
weights_probs = [w / sum(weights) for w in weights]

# DATA_PATH = "/kaggle/input/pii-detection-removal-from-educational-data"
DATA_PATH = "pll_data_detection/data"

# THRESHOLD = 0.20

In [2]:
weights_probs

[1.0]

# Install
https://www.kaggle.com/competitions/benetech-making-graphs-accessible/discussion/410130#2258335

In [3]:
!pip uninstall transformers datasets -y

Found existing installation: transformers 4.37.0
Uninstalling transformers-4.37.0:
  Successfully uninstalled transformers-4.37.0
Found existing installation: datasets 2.1.0
Uninstalling datasets-2.1.0:
  Successfully uninstalled datasets-2.1.0


In [4]:
# Not use pip install for match predict's notebook version
!python -m pip install -q --no-index --find-links=/kaggle/input/ner-transformer-library-notebook \
transformers \
seqeval \
datasets \
evaluate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
dask-cuda 23.8.0 require

# Import

In [5]:
import evaluate
import numpy as np
import polars as pl
import torch
import gc
import datasets
from scipy.special import softmax
from datasets import load_dataset
from tqdm.auto import tqdm
import transformers
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TokenClassificationPipeline,
    Trainer,
    TrainingArguments,
)

2024-04-13 01:21:33.360598: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 01:21:33.360737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 01:21:33.634871: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Check Environment

In [6]:
!python --version

Python 3.10.13


In [7]:
!nvidia-smi

Sat Apr 13 01:21:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
assert transformers.__version__ == '4.37.2'
assert datasets.__version__ == '2.16.1'
assert evaluate.__version__ == '0.4.1'

# Data Load

In [9]:
test_dataset = load_dataset(
    "json", data_files={"test": f"{DATA_PATH}/test.json"}, split="test"
)

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
# model = AutoModelForTokenClassification.from_pretrained(TRAINED_MODEL_PATH)
# tokenizer = AutoTokenizer.from_pretrained(TRAINED_MODEL_PATH)

# Prediction

In [11]:
def tokenize(example, tokenizer):
    text = []
    token_map = []
    
    idx = 0
    
    for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        
        text.append(t)
        token_map.extend([idx]*len(t))
        if ws:
            text.append(" ")
            token_map.append(-1)
            
        idx += 1
        
        
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=False)
       
    return {
        **tokenized,
        "token_map": token_map,
    }

In [12]:
tokenizer_tmp = AutoTokenizer.from_pretrained(TRAINING_MODEL_PATHS[0])
test_dataset = test_dataset.map(tokenize, fn_kwargs={"tokenizer": tokenizer_tmp}, num_proc=1)
# 現在は同じtokenizerを使ったモデルしか利用していないためエラーにはならないが、
# tokenizerが異なるモデルをensemubleする場合には問題になる。

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [13]:
# preds_final.shape
# (10, 2000)
# 10はデータ数
# Q. この2000という数字はどこから来たのか
# A. DataCollatorForTokenClassificationのpad_to_multiple_of=16にて、16の倍数にpaddingされるから。
#    len(test_dataset["input_ids"][9]) = 1994であり、これを16の倍数にpaddingすると一番近いのが2000である。

# Q. preds_final[0][-10:] -> array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
#    つまり、2000のうち、最後の10個を全て1(=B-NAME_STUDENT)と予測している。問題ないのか？
# A. 問題ない。if start_idx >= len(token_map):　breakを行っているため、
#    offset_mappingのstart_idxがtoken_mapより長くなる = paddingしているゾーン以降はpredictionに入らないようになっている。

In [14]:
del tokenizer_tmp
gc.collect()

88

In [15]:
# collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

In [16]:
# args = TrainingArguments(
#     ".", 
#     per_device_eval_batch_size=1, 
#     report_to="none",
# )
# trainer = Trainer(
#     model=model, 
#     args=args, 
#     data_collator=collator, 
#     tokenizer=tokenizer,
# )

In [17]:
# # pp なし
# # predictions = trainer.predict(test_dataset).predictions
# # pred_softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis = 2).reshape(predictions.shape[0],predictions.shape[1],1)
# # preds_final = predictions.argmax(-1)

# # ppあり

# predictions = trainer.predict(test_dataset).predictions
# pred_softmax = softmax(predictions, axis=2)

# preds = pred_softmax.argmax(-1)
# # preds_without_O = pred_softmax[:,:,:12].argmax(-1)
# preds_without_O = pred_softmax[:,:,1:].argmax(-1) + 1
# # O_preds = pred_softmax[:,:,12]
# O_pred_proba = pred_softmax[:,:,0]

# preds_final = np.where(O_pred_proba < THRESHOLD, preds_without_O , preds)

In [18]:
def make_pred_softmax(model_path: str, test_dataset: datasets.arrow_dataset.Dataset):
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
    
    args = TrainingArguments(
        ".", 
        per_device_eval_batch_size=1, 
        report_to="none",
        fp16=True,
    )
    
    trainer = Trainer(
        model=model, 
        args=args, 
        data_collator=collator, 
        tokenizer=tokenizer,
    )

    predictions = trainer.predict(test_dataset).predictions
    pred_softmax = softmax(predictions, axis=2)
    
    del model, trainer, collator, tokenizer, args, predictions
    torch.cuda.empty_cache()
    gc.collect()
    
    return pred_softmax

# pred_softmaxs = [
#     make_pred_softmax(model_path, test_dataset)
#     for model_path
#     in TRAINING_MODEL_PATHS
# ]

for idx, (weight_prob, model_path) in enumerate(zip(weights_probs, TRAINING_MODEL_PATHS)):
    if idx == 0:
        pred_softmax = weight_prob * make_pred_softmax(model_path, test_dataset)
    else:
        pred_softmax += weight_prob * make_pred_softmax(model_path, test_dataset)

# assert len(weights) == len(pred_softmaxs)

# pred_softmax_old = np.sum(
#     [
#         weight * one_pred_softmax
#         for weight, one_pred_softmax
#         in zip(
#             [w / sum(weights) for w in weights],
#             pred_softmaxs
#         )
#     ],
#     axis=0
# )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
# def postprocess_by_threshold(pred_softmax: np.ndarray, thr: float):
#     preds = pred_softmax.argmax(-1)
#     preds_without_O = pred_softmax[:,:,1:].argmax(-1) + 1
#     O_pred_proba = pred_softmax[:,:,0]

#     pred_final = np.where(O_pred_proba < thr, preds_without_O , preds)
    
#     del preds, preds_without_O, O_pred_proba
#     torch.cuda.empty_cache()
#     gc.collect()
    
#     return pred_final

# preds_final = postprocess_by_threshold(pred_softmax, THRESHOLD)
preds_final = pred_softmax.argmax(-1)

In [20]:
# del pred_softmax
# torch.cuda.empty_cache()
# gc.collect()

In [21]:
id2label = {
    0: "O",
    1: "B-NAME_STUDENT",
    2: "I-NAME_STUDENT",
    3: "B-EMAIL",
    4: "I-EMAIL",
    5: "B-USERNAME",
    6: "I-USERNAME",
    7: "B-ID_NUM",
    8: "I-ID_NUM",
    9: "B-PHONE_NUM",
    10: "I-PHONE_NUM",
    11: "B-URL_PERSONAL",
    12: "I-URL_PERSONAL",
    13: "B-STREET_ADDRESS",
    14: "I-STREET_ADDRESS",
}

In [ ]:
# triplets = []
document, token, label, token_str = [], [], [], []
for p, token_map, offsets, tokens, doc in zip(
    preds_final,
    test_dataset["token_map"],
    test_dataset["offset_mapping"],
    test_dataset["tokens"],
    test_dataset["document"]
):
    triplets = []
    for token_pred, (start_idx, end_idx) in zip(p, offsets):
        label_pred = id2label[token_pred]

        if start_idx + end_idx == 0:
            continue

        if token_map[start_idx] == -1:
            start_idx += 1

        # ignore "\n\n"
        # while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
        #    start_idx += 1
        
        # special tokenに\nを追加した場合は、\nのタイミングでstart_idxを=1したくないため
#         while start_idx < len(token_map) and tokens[token_map[start_idx]] == " ":
#             start_idx += 1

        # special tokenに\nを追加した場合は、\nのタイミングでstart_idxを=1したくないため
        while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace() and tokens[token_map[start_idx]] != "\n":
            start_idx += 1

        if start_idx >= len(token_map):
            break

        token_id = token_map[start_idx]

        # ignore "O" predictions and whitespace preds
        if label_pred != "O" and token_id != -1:
#         if (
#             label_pred not in ("O", "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM")
#             # label_pred not in ("O", "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM", "B-URL_PERSONAL", "I-URL_PERSONAL")
#             and token_id != -1
#         ):
            triplet = (label_pred, token_id, tokens[token_id])

            if triplet not in triplets:
                # 一つ前の(document_id, token_id)と今の(document_id, token_id)が同じ場合は、tripletに追加しない
                if len(triplets) >= 1:
                    if document[-1] == doc and token[-1] == token_id:
                        continue
                document.append(doc)
                token.append(token_id)
                label.append(label_pred)
                token_str.append(tokens[token_id])
                triplets.append(triplet)

> /tmp/ipykernel_34/3116561652.py(13)<module>()
     11     for token_pred, (start_idx, end_idx) in zip(p, offsets):
     12         import pdb; pdb.set_trace()
---> 13         label_pred = id2label[token_pred]
     14 
     15         if start_idx + end_idx == 0:



ipdb>  token_pred


0


ipdb>  start_idx


0


...

'O'


ipdb>  n


> /tmp/ipykernel_34/3116561652.py(16)<module>()
     14 
     15         if start_idx + end_idx == 0:
---> 16             continue
     17 
     18         if token_map[start_idx] == -1:



ipdb>  n


> /tmp/ipykernel_34/3116561652.py(11)<module>()
      9 ):
     10     triplets = []
---> 11     for token_pred, (start_idx, end_idx) in zip(p, offsets):
     12         import pdb; pdb.set_trace()
     13         label_pred = id2label[token_pred]

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
> /tmp/ipykernel_34/3116561652.py(13)<module>()
     11     for token_pred, (start_idx, end_idx) in zip(p, offsets):
     12         import pdb; pdb.set_trace()
---> 13         label_pred = id2label[token_pred]
     14 
     15         if start_idx + end_idx == 0:



In [23]:
output_df = pl.DataFrame(
    [document, token, label, token_str], schema=["document", "token", "label", "token_str"]
)

In [24]:
# from spacy.lang.en import English

# nlp = English()

# def find_span(target: list[str], document: list[str]) -> list[list[int]]:
#     idx = 0
#     spans = []
#     span = []

#     for i, token in enumerate(document):
#         if token != target[idx]:
#             idx = 0
#             span = []
#             continue
#         span.append(i)
#         idx += 1
#         if idx == len(target):
#             spans.append(span)
#             span = []
#             idx = 0
#             continue

#     return spans

In [25]:
# import re
# from typing import Optional


# def get_rulebase(regex: re.Pattern, data, name: str) -> Optional[list[dict]]:
#     output = []
#     matches = regex.findall(data["full_text"])

#     # NOTE: find_spanにおいて、同じ単語を全て見つけてしまうため、重複をなくす
#     matches = list(dict.fromkeys(matches))

#     if not matches:
#         return None

#     matched_spans = []
#     for match in matches:
#         target = [t.text for t in nlp.tokenizer(match)]
#         matched_spans.append(find_span(target, data["tokens"]))

#     for matched_span in matched_spans:
#         for one_token_span in matched_span:
#             for intermediate, token_idx in enumerate(one_token_span):
#                 if intermediate == 0:
#                     prefix = "B"
#                 else:
#                     prefix = "I"

#                 output.append(
#                     {
#                         "document": data["document"],
#                         "token": token_idx,
#                         "label": f"{prefix}-{name}",
#                         "token_str": data["tokens"][token_idx],
#                     }
#                 )

#     return output


# email_regex = re.compile(r"[\w.+-]+@[\w-]+\.[\w.-]+")
# phone_num_regex = re.compile(r"(\(\d{3}\)\d{3}\-\d{4}\w*|\d{3}\.\d{3}\.\d{4})\s")
# # url_regex = re.compile(
# #     r"https?://(?:www\.)?[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+){1,2}/(?:[a-zA-Z0-9-]+/)*(?:[a-zA-Z0-9-]+\.html|.*\.php|.*\.asp|.*\.jsp|\?v=[a-zA-Z0-9-_]+|.*\.htm|user/[a-zA-Z0-9-_]+|watch\?v=[a-zA-Z0-9-_]+|[a-zA-Z0-9-_]+|\?v=[a-zA-Z0-9-_]+|[a-zA-Z0-9-_]+|)?"
# # )

# emails = []
# phone_nums = []
# # urls = []

# for _data in test_dataset:
#     # email
#     if match := get_rulebase(email_regex, _data, "EMAIL"):
#         emails.extend(match)

#     # phone number
#     if match := get_rulebase(phone_num_regex, _data, "PHONE_NUM"):
#         phone_nums.extend(match)

#     # URL
# #     if match := get_rulebase(url_regex, _data, "URL_PERSONAL"):
# #         urls.extend(match)

# # pp_data = [emails, phone_nums, urls]
# # pp_data = [emails, phone_nums]

In [26]:
# # if emails != [] or phone_nums != [] or urls != []:
# if emails != [] or phone_nums != []:
#     postprocess_df = pl.concat(
#         [pl.DataFrame(val) for val in pp_data if val != []]
#     )
#     output_df = pl.concat([output_df, postprocess_df])
    
#     del postprocess_df
#     gc.collect()

In [27]:
# output_df = pl.DataFrame(
#     [document, token, label, token_str], schema=["document", "token", "label", "token_str"]
# )

output_df = (
    output_df
    .select(
        pl.col(["document", "token", "label"])
    )
    .sort(
        ["document", "token"]
    )
    .with_row_index(
        name="row_id"
    )
)

In [28]:
if output_df.height < 30:
    display(output_df)

row_id,document,token,label
u32,i64,i64,str
0,7,9,"""B-NAME_STUDENT…"
1,7,10,"""I-NAME_STUDENT…"
2,7,482,"""B-NAME_STUDENT…"
3,7,483,"""I-NAME_STUDENT…"
4,7,741,"""B-NAME_STUDENT…"
5,7,742,"""I-NAME_STUDENT…"
6,10,0,"""B-NAME_STUDENT…"
7,10,1,"""I-NAME_STUDENT…"
8,10,464,"""B-NAME_STUDENT…"


In [29]:
output_df.write_csv("submission.csv")